In [43]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

## Scrap Contents for the required URLs
City,country,population,coordination

In [44]:
cities = [
            "London"
            ,"Paris"
            ,"Berlin"
            ,"Moscow"
            ,"Istanbul"
            ,"Barcelona"
            ,'Saint Petersburg'
            ,'Madrid'
            ,'Kyiv'
            ,'Rome'
            ,'Bucharest'
            ,'Minsk'
            ,'Vienna']
soups=[]
url_wiki = "https://en.wikipedia.org/wiki/"
headers = {'Accept-Language': 'en-US,en;q=0.8'}

for city in cities:
    response = requests.get(url_wiki+city, headers = headers)
    if response.status_code == 200:
        soups.append(BeautifulSoup(response.content, "html.parser"))
    else:
        print(f"city {city} doesn't have a wiki")
        soups.append("")

In [45]:
country = []
coordinate = []
pop_des = []
pop_value = []

In [46]:
for i,city in enumerate(cities):
    table_name = "table.infobox.ib-settlement.vcard > tbody "
    for trs in soups[i].select(table_name + " tr"):
        #Find Country
        if trs and ( "Country" in trs.get_text()):
           country.append(trs.findNext('td').get_text().strip())
            
        # Find Coordinate
        elif trs and ( "Coordinates" in trs.get_text()):
            coordinate.append(trs.findNext('span').get_text().split("/")[1].strip().replace(u'\ufeff', ''))
        # Find Population
        elif trs and ( "Population" in trs.get_text()):
            #description
            pop_des.append(trs.findNext('th').get_text().split("[")[0].replace(u'\xa0', ' '))
            #population value
            x=trs.findNext('td').get_text()
            x=x.replace(",","").split("[")[0]
            try:
                x= int(x)
                pop_value.append(x)
            except:
                pop_value.append("NotAvailable")
            break

## Scrap country code from URLs

In [47]:
# url_wiki = "https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes"
# headers = {'Accept-Language': 'en-US,en;q=0.8'}

# response = requests.get(url_wiki, headers = headers)
# if response.status_code == 200:
#     soup = BeautifulSoup(response.content, "html.parser")
# else:
#     print(f"Wrong URL!")

In [61]:
# country_iso = []
# table_class = "table.wikitable.sortable > tbody "
# for count in country:
#     for trs in soup.select(table_class + " tr"):
#         if trs and ( count in trs.get_text()):
#         #Find Country code
#             country_iso.append(trs.select("td")[3].get_text().replace("\n",""))
#             break

In [64]:
country

['England',
 'France',
 'Germany',
 'Russia',
 'Turkey',
 'Spain',
 'Russia',
 'Spain',
 'Ukraine',
 'Italy[a]',
 'Romania',
 'Belarus',
 'Austria']

In [65]:
country_iso = ['GB','FR', 'DE', 'RU', 'TR','ES', 'RU', 'ES', 'UA','IT', 'RO', 'BY', 'AT']

In [66]:
len(cities)==len(country) == len(coordinate) == len(pop_des) == len(pop_value) == len(country_iso)

True

In [60]:
len(pop_value)

13

In [67]:
df = pd.DataFrame(
    {"city": cities,
     "country": country,
     "country_iso": country_iso,
     "coordination": coordinate,
     "pop_des": pop_des,
     "pop_value":pop_value
    }
)

## Create city_id

In [68]:
df

,city,country,country_iso,coordination,pop_des,pop_value
0,London,England,GB,51.50722°N 0.12750°W,Population (2021 except where stated),8799800
1,Paris,France,FR,48.856613°N 2.352222°E,Population (Jan. 2019),2165423
2,Berlin,Germany,DE,52.52000°N 13.40500°E,Population (2021),3677472
3,Moscow,Russia,RU,55.75583°N 37.61722°E,Population (2021 Census),13010112
4,Istanbul,Turkey,TR,41.01361°N 28.95500°E,Population (31 December 2021),15840900
5,Barcelona,Spain,ES,41.383°N 2.183°E,Population (2018),1620343
6,Saint Petersburg,Russia,RU,59.93750°N 30.30861°E,Population,5351935
7,Madrid,Spain,ES,40.41667°N 3.70250°W,Population (2018),3223334
8,Kyiv,Ukraine,UA,50.45000°N 30.52333°E,Population (1 January 2021),2962180
9,Rome,Italy[a],IT,41.89333°N 12.48278°E,Population (31 December 2019),NotAvailable


In [69]:
df = df.reset_index()

In [70]:
df = df.rename(columns={"index": "city_id"})

In [71]:
df

,city_id,city,country,country_iso,coordination,pop_des,pop_value
0,0,London,England,GB,51.50722°N 0.12750°W,Population (2021 except where stated),8799800
1,1,Paris,France,FR,48.856613°N 2.352222°E,Population (Jan. 2019),2165423
2,2,Berlin,Germany,DE,52.52000°N 13.40500°E,Population (2021),3677472
3,3,Moscow,Russia,RU,55.75583°N 37.61722°E,Population (2021 Census),13010112
4,4,Istanbul,Turkey,TR,41.01361°N 28.95500°E,Population (31 December 2021),15840900
5,5,Barcelona,Spain,ES,41.383°N 2.183°E,Population (2018),1620343
6,6,Saint Petersburg,Russia,RU,59.93750°N 30.30861°E,Population,5351935
7,7,Madrid,Spain,ES,40.41667°N 3.70250°W,Population (2018),3223334
8,8,Kyiv,Ukraine,UA,50.45000°N 30.52333°E,Population (1 January 2021),2962180
9,9,Rome,Italy[a],IT,41.89333°N 12.48278°E,Population (31 December 2019),NotAvailable


## Fix Coordinations

In [88]:
def ConvertCoord(row):
    #convert lat and long
    lat,long = row.coordination.replace("°E","").replace("°N","").split(" ")
    if lat.find("°S") != -1 :
        lat="-" + lat.replace("°S","")
    if long.find("°W") != -1 :
        long="-" + long.replace("°W","")
    try:
        lat= float(lat)
        long= float(long)
    except:
        print("ERROR, COORDINATES are not Convertable")
    return (lat,long)

In [97]:
df["lat"]=df.apply(ConvertCoord,axis=1,result_type="expand")[0]

In [99]:
df["long"]=df.apply(ConvertCoord,axis=1,result_type="expand")[1]

In [100]:
df

,city_id,city,country,country_iso,coordination,pop_des,pop_value,lat,long
0,0,London,England,GB,51.50722°N 0.12750°W,Population (2021 except where stated),8799800,51.507220,-0.127500
1,1,Paris,France,FR,48.856613°N 2.352222°E,Population (Jan. 2019),2165423,48.856613,2.352222
2,2,Berlin,Germany,DE,52.52000°N 13.40500°E,Population (2021),3677472,52.520000,13.405000
3,3,Moscow,Russia,RU,55.75583°N 37.61722°E,Population (2021 Census),13010112,55.755830,37.617220
4,4,Istanbul,Turkey,TR,41.01361°N 28.95500°E,Population (31 December 2021),15840900,41.013610,28.955000
5,5,Barcelona,Spain,ES,41.383°N 2.183°E,Population (2018),1620343,41.383000,2.183000
6,6,Saint Petersburg,Russia,RU,59.93750°N 30.30861°E,Population,5351935,59.937500,30.308610
7,7,Madrid,Spain,ES,40.41667°N 3.70250°W,Population (2018),3223334,40.416670,-3.702500
8,8,Kyiv,Ukraine,UA,50.45000°N 30.52333°E,Population (1 January 2021),2962180,50.450000,30.523330
9,9,Rome,Italy[a],IT,41.89333°N 12.48278°E,Population (31 December 2019),NotAvailable,41.893330,12.482780


## Split data to 2 dfs

In [101]:
cities_df = df.drop(["pop_des","pop_value"],axis=1)

In [102]:
cities_df

,city_id,city,country,country_iso,coordination,lat,long
0,0,London,England,GB,51.50722°N 0.12750°W,51.507220,-0.127500
1,1,Paris,France,FR,48.856613°N 2.352222°E,48.856613,2.352222
2,2,Berlin,Germany,DE,52.52000°N 13.40500°E,52.520000,13.405000
3,3,Moscow,Russia,RU,55.75583°N 37.61722°E,55.755830,37.617220
4,4,Istanbul,Turkey,TR,41.01361°N 28.95500°E,41.013610,28.955000
5,5,Barcelona,Spain,ES,41.383°N 2.183°E,41.383000,2.183000
6,6,Saint Petersburg,Russia,RU,59.93750°N 30.30861°E,59.937500,30.308610
7,7,Madrid,Spain,ES,40.41667°N 3.70250°W,40.416670,-3.702500
8,8,Kyiv,Ukraine,UA,50.45000°N 30.52333°E,50.450000,30.523330
9,9,Rome,Italy[a],IT,41.89333°N 12.48278°E,41.893330,12.482780


In [103]:
population_df = df.drop(["city","country","coordination","country_iso","lat","long"],axis=1)

In [104]:
population_df

,city_id,pop_des,pop_value
0,0,Population (2021 except where stated),8799800
1,1,Population (Jan. 2019),2165423
2,2,Population (2021),3677472
3,3,Population (2021 Census),13010112
4,4,Population (31 December 2021),15840900
5,5,Population (2018),1620343
6,6,Population,5351935
7,7,Population (2018),3223334
8,8,Population (1 January 2021),2962180
9,9,Population (31 December 2019),NotAvailable


## Export Final dfs

In [105]:
cities_df.to_pickle("out_dfs\cities_2.pkl")

In [106]:
population_df.to_pickle("out_dfs\population_2.pkl")